# Libraries and functions

In [1]:
# pdf webscrapping and text extraction

# get the pdf files or url from the web
import requests

# for tree traversal scraping in webpage
from bs4 import BeautifulSoup

# input output operations
import io

# getting information from the pdfs
#from PyPDF2 import PdfReader

#!pip install xhtml2pdf requests
#!pip install lxml

# for converting html to pdf
import regex as re
import pandas as pd
from urllib.request import urlopen
from urllib import request as urllib2


/Users/yasminsarkhosh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



# Extraction: by url


Front page with 730 links, where each link directs you to another page with a specific research paper.
Code below extracts text from the second webpage. However, it is not able to extract text from the pdf that are linked in the second page. There are 2 different links for downloading the same pdf. 
- DOI and 
- ShareIt


```<p>
DOI: 
<a href="https://doi.org/10.1007/978-3-031-43990-2_43">https://doi.org/10.1007/978-3-031-43990-2_43</a>
</p>


<p>
SharedIt: 
<a href="https://rdcu.be/dnwLY">https://rdcu.be/dnwLY</a>
<br><br>
</p>

In [ ]:
import requests 
from bs4 import BeautifulSoup 
import regex as re
from urllib.request import urlopen
from urllib import request as urllib2

# specify the URL of the archive here 
archive_url = "http://conferences.miccai.org/2023/papers/"
url = "http://conferences.miccai.org"


list_of_pdfs = []
# create response object 
r = requests.get(archive_url) 

# create beautiful-soup object 
soup = BeautifulSoup(r.content,'html')

# find all links on web-page 
links = soup.findAll('a') 

# filter links ending with .html 
list_of_urls = [url + link['href'] for link in links if link['href'].endswith('html')] 
# print(len(paper_links)) gives us 730 papers in total


# now that we have all the links to the papers, we can go through each paper and find the pdf link
for link in list_of_urls:
	# create response object per link from the total list of links
	r = requests.get(link) 
	soup = BeautifulSoup(r.content,'html')

	# find all urls with 'rdcu.be' which contains the specific paper
	links = soup.findAll(attrs={'href': re.compile("rdcu.be")})
	pdfs = [link['href'] for link in links]	
	list_of_pdfs.extend(pdfs)

print(len(list_of_pdfs))


In [ ]:
for link in list_of_pdfs:
	# create response object per link from the total list of links
	r = requests.get(link) 
	soup = BeautifulSoup(r.content,'html')
	# find all urls with 'rdcu.be' which contains the specific paper
	links = soup.findAll(attrs={'href': re.compile("/content/pdf")})
	final_list = [link['content'] for link in links]	

print(len(final_list))  

In [ ]:
for pdf in list_of_pdfs:
    filename = pdf.split("/")[-1]
    res = requests.get(pdf)
    pdf = open("pdfs/" + filename, 'wb')
    pdf.write(res.content)
    pdf.close() 

# Extracting papers into dataframe


In [ ]:
from bs4 import BeautifulSoup
import re

part1 = open('/Users/yasminsarkhosh/Documents/micca2023part2.doc', encoding = "ISO-8859-1")
soup = BeautifulSoup(part1, 'html.parser')
#parsing the document

In [ ]:
def has_doi(href):
    return href and re.compile("chapter/").search(href)

list_of_doi = soup.find_all(href=has_doi)

In [ ]:
list_of_doi

In [ ]:
#getting the titles and the doi's from above function

titles = []
doi_str = []

for element in list_of_doi:
    titles.append(element.get_text())
    string = str(element)
    first_substring = '/chapter'
    second_substring ='">'
    doi_str.append(string[(string.find(first_substring)):string.find(second_substring)])

In [ ]:
titles

In [ ]:
## now want to find authors - YES!
authors = []

authors = soup.find_all("li", class_="c-author-list__item")

In [ ]:
#keeping only the author names
authors_str = []
for element in authors:
    string = str(element)
    first_substring = 'item">'
    second_substring ='</li>'
    authors_str.append(string[(string.find(first_substring)+6):string.find(second_substring)])

In [ ]:
authors_str

In [ ]:
#next up is page numbers
#line to find:
#<div class="c-meta"><span class="c-meta__item u-display-inline-block" data-test="page-number">Pages 1-9</span>

page_numbers = []

page_numbers= soup.find_all('div', class_ = "c-meta")
#page_numbers = page_numbers[3:] #removing the first two pages that are not relevant/chapters
page_numbers = page_numbers[3:] #removing the first two pages that are not relevant/chapters

In [ ]:
print(len(page_numbers))
print(len(authors_str))
print(len(titles))


In [ ]:
# <h3 class="c-card__title" data-test="back-matter">Back Matter</h3>
            
           # <div class="c-meta"><span class="c-meta__item u-display-inline-block" data-test="page-number">Pages 623-626</span>
back_matter = []
back_matter = soup.find_all(">Back Matter<")

back_matter

In [ ]:
#strips it back to simply the pages
page_numbers_str = []

for element in page_numbers:
    string = str(element)
    first_substring = 'Pages'
    second_substring ='</span>'
    string = string[(string.find(first_substring)):string.find(second_substring)]
    if len(string) > 0: #ensures I don't add an empty string to the list
        page_numbers_str.append(string)

page_numbers_str 

In [ ]:
#a check that I've found the same amount of authors, titles, dois and page numbers
if len(page_numbers_str) == len(authors_str) == len(titles) == len(doi_str):
    print("true")
else:
    print("You have an error!")

In [ ]:
#now want to combine everything in a pandas dataframe

import pandas as pd
#need to add the year (probably a better way to do this, but should be fine)
year_of_pub = []
for element in titles:
    year_of_pub.append(2023)

In [ ]:
data = {'Title': titles,
        'Authors': authors_str,
        'Page numbers' : page_numbers_str,
        'DOI': doi_str,
        'Year of publication' : year_of_pub
        }

df = pd.DataFrame(data)

df.to_csv('miccai_2023_part2.csv')
#not sure why it seems to put "" around only the authors in the csv file, but good enough to start! 
#Now want to combine this into a more pretty script that I can then feed all the pages into it, will need to do some 
#combining at the end, adding everything together

# Main mining function 

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
#helper function to find lines in html document with DOI and titles of articles
def has_doi(href):
    return href and re.compile("chapter/").search(href)

In [11]:
#main mining function returning the initial dataframe 

def mining(html_doc, year, current_page, all_pages, part):
    #opening the html document (copy pasted and saved as a .doc file)
    doc = open(html_doc, "r", encoding = "ISO-8859-1") 
    soup = BeautifulSoup(doc, 'html.parser' )

    list_of_doi = soup.find_all(href=has_doi)
        
    #getting the titles and the doi's from list generated helper function

    titles = []
    doi_str = []

    for element in list_of_doi:
        titles.append(element.get_text()) #returns the titles as the only text in the list
        string = str(element)
        first_substring = '/chapter'
        second_substring ='">'
        #separates out the DOIS (added the +9 to remove /chapter/ from the beginning of all DOIS)
        doi_str.append(string[(string.find(first_substring)+9):string.find(second_substring)]) 
            
    ## now the lines containing author are found
    authors = soup.find_all("li", class_="c-author-list__item")
        
    #keeping only the author names
    authors_str = []
    for element in authors:
        string = str(element)
        first_substring = 'item">'
        second_substring ='</li>'
        authors_str.append(string[(string.find(first_substring)+6):string.find(second_substring)])
            
    #now the lines containing page numbers are found
    page_numbers= soup.find_all('div', class_ = "c-meta")
        
    #keeping only the page numbers
    page_numbers_str = []

    for element in page_numbers:
        string = element.get_text()[6:-1] #removes the white spaces and "Pages "
        both = string.split("-")
        if 'x' not in string: #filtering out front matters
            try: 
                if int(both[1])-int(both[0]) > 1: 
                    page_numbers_str.append(string)
            except:
                if "C" in string or "E" in string: #including corrections and erratum, are removed later
                    page_numbers_str.append(string)
    #filtering out back matters, only an issue in 2021 
    if year == 2023 and int(current_page) == int(all_pages):
        page_numbers_str = page_numbers_str[:-1]
            
    #need to create a list of the year of publication to add to dataframe 
    year_of_pub = []
    for element in titles:
        year_of_pub.append(year)
    
    #will add the part of the publication to the dataframe as well
    part_of_pub = []
    for element in titles:
        part_of_pub.append(part)
        
    
    #creating the column names and content for the dataframe        
    data = {'Title': titles,
        'Authors': authors_str,
        'Page numbers' : page_numbers_str,
        'DOI': doi_str,
        'Year of publication' : year_of_pub,
        'Part of publication' : part_of_pub       }

    df = pd.DataFrame(data)

    return df 

In [15]:
#helper function to combine all df together
def data_together(data, year):
    combined_frame = pd.concat(data, ignore_index = True, sort = False)
    if year == 2023:
        combined_frame.drop(combined_frame[combined_frame["Title"].str.contains("Correction to")].index, inplace=True)
    combined_frame.to_csv('database_miccai_'+ str(year) +'.csv')
   
    return combined_frame

In [21]:
#saving all 2012 together as one with the helper function above
miccai =['/Users/yasminsarkhosh/Documents/miccai2023 papers/miccai2023part1 page 1 of 4.doc',
         '/Users/yasminsarkhosh/Documents/miccai2023 papers/miccai2023part1 page 2 of 4.doc',
         '/Users/yasminsarkhosh/Documents/miccai2023 papers/miccai2023part1 page 3 of 4.doc',
         '/Users/yasminsarkhosh/Documents/miccai2023 papers/miccai2023part1 page 4 of 4.doc',

         '/Users/yasminsarkhosh/Documents/miccai2023 papers/miccai2023part2 page 1 of 4.doc',
         '/Users/yasminsarkhosh/Documents/miccai2023 papers/miccai2023part2 page 2 of 4.doc',
         '/Users/yasminsarkhosh/Documents/miccai2023 papers/miccai2023part2 page 3 of 4.doc']


data = []
for element in miccai:
    print(element[-10], element[-5], element[-17])  
    data.append(mining(element, 2023, element[-10], element[-5], element[-17]))

data_together(data, 2023)

1 4 1
2 4 1
3 4 1
4 4 1


ValueError: All arrays must be of the same length

In [7]:
miccai =['/Users/yasminsarkhosh/Documents/micca2023part2.doc']

data = []
for element in miccai:
    data.append(mining(element, 2023, element[-5]))


data_together(data, 2023) 

TypeError: mining() missing 2 required positional arguments: 'all_pages' and 'part'

In [ ]:
ß

In [ ]:
  '/Users/yasminsarkhosh/Documents/micca2023part3.doc']